<a href="https://colab.research.google.com/github/VladShajdulin/OTUS_project/blob/main/OTUS_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import json
import regex
import torch
from torch import Tensor
from sentence_transformers import CrossEncoder
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
torch.manual_seed(142)
path = '/content/drive/MyDrive/Colab Notebooks/OTUS/final_project/articles_links.json'
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else 'cpu'
print(device)

cpu


In [ ]:
all_data = pd.read_json(path)
df = all_data[['title', 'link', 'annot']]
all_data

,title,author,date,imp_ind,views,annot,tags,link
0,"Ученые обнаружили еще одну причину, по которой...",Игорь Байдов,2025-10-10,5.2,21600,Голые землекопы живут почти 40 лет — невероятн...,"[Биология, голые землекопы, ДНК, долголетие]",https://naked-science.ru/article/biology/uchen...
1,"Бактерии создали из себя батарейку, чтобы разр...",Evgenia Vavilova,2025-10-10,4.2,7800,Они использовали содержащееся в почве железо. ...,"[Биология, аккумулятор, бактерии, биопленки]",https://naked-science.ru/article/biology/bacte...
2,Бактерии и добавки пероксида кальция сделают д...,Губкинский университет,2025-10-10,4.8,149,Ученые РГУ нефти и газа (НИУ) имени И.М. Губки...,"[Губкинский университет, арктика, бактерии, ка...",https://naked-science.ru/article/column/bakter...
3,Российские физики предложили искать темную мат...,ФизТех,2025-10-10,5.1,1000,Большой коллектив российских ученых из ведущих...,"[ФизТех, вселенная, квантование гравитации, те...",https://naked-science.ru/article/column/temnuy...
4,Белорусские ученые вырастили кожу из грибов и ...,Unitsky String Technologies Inc.,2025-10-10,4.5,430,Специалисты UST Inc. разработали технологию по...,"[Unitsky String Technologies Inc., биоматериал...",https://naked-science.ru/article/column/beloru...
...,...,...,...,...,...,...,...,...
8267,В Испании нашли свой Стоунхендж,Алиса Гаджиева,2022-08-22,4.3,1300,"На поле, предназначенном для выращивания авока...","[Антропология, археология, древние люди, Испания]",https://naked-science.ru/article/anthropology/...
8268,Астрономы оценили максимально возможное число ...,Сергей Васильев,2022-08-22,5.2,8600,"Ученые подсчитали, что без больших изменений о...","[Астрономия, естественный спутник, Луна, спутн...",https://naked-science.ru/article/astronomy/mak...
8269,Рецепт сознания: могут ли люди и роботы мечтат...,Денис Тулинов,2022-08-22,5.0,7600,В XXI веке вопрос о природе сознания станет од...,"[Технологии, искусственный интеллект, машинное...",https://naked-science.ru/article/hi-tech/retse...
8270,У берегов Западной Африки нашли древний ударны...,Мария Азарова,2022-08-21,6.5,4700,"Ученые допускают, что 66 миллионов лет назад с...","[Геология, астероид, гибель динозавров, диноза...",https://naked-science.ru/article/geology/udarn...


In [ ]:
def clean_text(text, pattern=r'[^\w\p{P}]+'):
  pattern = regex.compile(pattern)
  text = pattern.sub(' ', text)

  return regex.sub(r'\s+', ' ', text).strip()

In [ ]:
texts = [clean_text(t) for t in df['annot']]
passages = [f'passage: {t}' for t in texts]
texts[:5]

['Голые землекопы живут почти 40 лет — невероятный срок для грызуна размером с мышь. Недавнее открытие китайских ученых показало, что долголетие этим животным обеспечивает необычная версия одного белка, который не просто защищает организм, а буквально чинит его изнутри — ускоряет восстановление ДНК. Новые данные могут помочь продлить и человеческую жизнь.',
 'Они использовали содержащееся в почве железо. Днем бактерии захватывали фотоны, ночью — боролись с мешающими им жить соединениями.',
 'Ученые РГУ нефти и газа (НИУ) имени И.М. Губкина разработали новый метод обезвреживания нефтесодержащих отходов в Арктической зоне России. Он основан на комбинированном использовании бактериальных препаратов с добавками пероксида кальция, источника кислорода при обезвреживании отходов с использованием геотубных технологий. Технология успешно прошла полевые испытания.',
 'Большой коллектив российских ученых из ведущих физических институтов Москвы, Нижнего Новгорода, Сарова и Санкт-Петербурга предста

# 1. Семантический поиск
Для начала сделаем обычный семантический поиск на основе концепции "Retrieve & Re-rank". Посчитаем эмбеддинги для всех аннотаций статей, отберем из них топ 100, и затем посчитаем новые ранги для этого топа, в качестве реузьтата будет 10 наиболее релевантных статей.

In [ ]:
model = SentenceTransformer('d0rj/e5-base-en-ru', device=device)
embeddings = model.encode(
    passages,
    convert_to_tensor=True,
    show_progress_bar=True,
    device=device,
    normalize_embeddings=True
)
torch.save(embeddings, '/content/drive/MyDrive/Colab Notebooks/OTUS/final_project/embeddings.pt')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/259 [00:00<?, ?it/s]

In [ ]:
embeddings = torch.load('/content/drive/MyDrive/Colab Notebooks/OTUS/final_project/embeddings.pt', map_location=device)
embeddings.shape

torch.Size([8272, 768])

In [ ]:
model = SentenceTransformer('d0rj/e5-base-en-ru', device=device)
reranker = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1', device=device)

def semantic_search(query, retriever, reranker, embeddings=embeddings, texts=texts, top_n=100, result=10):
  request = retriever.encode(
      f'query: {query}',
      convert_to_tensor=True,
      device=device,
      normalize_embeddings=True
  )

  scores = embeddings @ request
  raiting = torch.argsort(scores, descending=True)
  raiting = raiting[:top_n].tolist()
  documents = [texts[i] for i in raiting]

  new_top = reranker.rank(query, documents)
  ranks = [r['corpus_id'] for r in new_top]
  new_rating = [raiting[i] for i in ranks]

  return new_rating[:result]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [ ]:
query = 'стандартная модель'
search_result = semantic_search(query, retriever=model, reranker=reranker, top_n=50)
df.iloc[search_result, :2]

,title,link
6956,Ученые создали подробнейшую карту распределени...,https://naked-science.ru/article/astronomy/mat...
547,Масса нейтрино оказалась не связана с темной м...,https://naked-science.ru/article/physics/neutr...
2845,Метод пермских ученых предсказал поведение сте...,https://naked-science.ru/article/column/metod-...
1137,Российские ученые научились справедливо делить...,https://naked-science.ru/article/column/sprave...
8002,Ученые разработали нейрокомпьютерную модель дл...,https://naked-science.ru/article/biology/uchen...
6221,Редкий распад бозона Хиггса может закрыть поис...,https://naked-science.ru/article/physics/raspa...
7793,В ЛЭТИ разработали модель для изучения магмы п...,https://naked-science.ru/article/column/v-leti...
5798,GPT-3 выполнил тест на мышление аналогиями луч...,https://naked-science.ru/article/hi-tech/gpt3-...
1587,"Разработана модель, учитывающая сразу нескольк...",https://naked-science.ru/article/column/parame...
1254,Искусственный интеллект поможет точнее прогноз...,https://naked-science.ru/article/column/progno...
